# CS 109A/STAT 121A/AC 209A/CSCI E-109A: 
# Final Project - 2017

**Harvard University**<br/>
**Fall 2017**<br/>
**Instructors**: Pavlos Protopapas, Kevin Rader, Rahul Dave, Margo Levine<br/>
**Leading TF**: Albert Wu<br/>
**Project Group #**: 16 (Sports)

---

In [2]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import IFrame, HTML
import requests
from bs4 import BeautifulSoup

In [3]:
box1_req = requests.get("https://www.basketball-reference.com/boxscores/201710170CLE.html")
box1_page = box1_req.text
box1_soup = BeautifulSoup(box1_page, 'html.parser')

In [16]:
box1_tables = box1_soup.find('div', id="all_box_bos_basic")

In [20]:
box1_rows = box1_tables.find('table', 'stats_table', 'sortable').find('tbody').find_all('tr')

In [28]:
players = []
for row in box1_rows:
    players.append(row.find('th').get_text())
players.remove('Reserves')

In [42]:
players

['Jaylen Brown',
 'Kyrie Irving',
 'Jayson Tatum',
 'Al Horford',
 'Gordon Hayward',
 'Marcus Smart',
 'Terry Rozier',
 'Aron Baynes',
 'Semi Ojeleye',
 'Shane Larkin',
 'Abdel Nader',
 'Daniel Theis']

In [126]:
game1_dict = {}
count_start = 0
for player in players:
    if count_start < 5:
        game1_dict[player] = [1]
        count_start += 1
    else:
        game1_dict[player] = [0]

In [127]:
game1_dict

{'Abdel Nader': [0],
 'Al Horford': [1],
 'Aron Baynes': [0],
 'Daniel Theis': [0],
 'Gordon Hayward': [1],
 'Jaylen Brown': [1],
 'Jayson Tatum': [1],
 'Kyrie Irving': [1],
 'Marcus Smart': [0],
 'Semi Ojeleye': [0],
 'Shane Larkin': [0],
 'Terry Rozier': [0]}

In [62]:
game1_req = requests.get("https://www.basketball-reference.com/boxscores/pbp/201710200PHI.html")
game1_page = game1_req.text
game1_soup = BeautifulSoup(game1_page, 'html.parser')

In [63]:
gmae1_table = game1_soup.find('table', 'stats_table', 'sortable')

In [67]:
game1_rows = gmae1_table.find_all('tr')

In [107]:
len(game1_rows)

520

In [ ]:
# Set up variables:
game1_dict['time_left'] = []

game1_dict['home_make'] = []
game1_dict['home_assist'] = []
game1_dict['home_fouled'] = []


game1_dict['road_score'] = []
game1_dict['home_score'] = []
game1_dict['score_diff'] = []

In [ ]:
quarter_str = ['Start of 1st', 'Start of 2nd', 'Start of 3rd', 'Start of 4th']
quarter_mul = [1, 2, 3, 4]
for row in game1_rows:
    if len(row.find_all('td')) == 6:
        row_info = row.find_all('td')
        
        # Calculate Time Left
        time_str = row_info[0].get_text().split(':')
        time_min = float(time_str[0])
        time_s = float(time_str[1])
        time_adj = time_min*mul + time_s
        game1_dict['time_left'].append(1 - time_adj/(48*60))
        
        # Road Team Possesion
        home_possesion = row_info[1].get_text()
        if home_possesion != '\xa0':
            if 'makes' in home_possesion:
                game1_dict['home_make'].append(1)
                if 'assist' in home_possesion:
                    game1_dict['home_assist'].append(1)
                game1_dict['home_fouled'].append(0)
            else:
                game1_dict['home_make'].append(0)
                game1_dict['home_assist'].append(0)
                if 'Shooting foul' in home_possesion:
                    game1_dict['home_fouled'].append(1)
            
        # Score
        score_str = row_info[3].split('-')
        game1_dict['road_score'].append(int(score_str[0]))
        game1_dict['home_score'].append(int(score_str[1]))
        game1_dict['score_diff'].append(home_score - road_score)
        
        # Home Tram Possesion
        if row_info[5].get_text() != '\xa0':
            
    else:
        for ind in range(len(quarter_str)):
            if quarter_str[ind] in row.get_text():
                mul = quarter_mul[ind]
                break

In [128]:
game1_rows[4].find_all('td')

[<td>11:40.0</td>,
 <td> </td>,
 <td> </td>,
 <td class=" center">0-0</td>,
 <td class=""> </td>,
 <td class=""><a href="/players/c/covinro01.html">R. Covington</a> misses 2-pt shot from 11 ft</td>]

In [130]:
game1_rows[4].find_all('td')[5].get_text()

'R. Covington misses 2-pt shot from 11 ft'

In [33]:
rown = game1_rows[3]
rown.find_all('td')[1].find_all('a')[0].get_text()

'J. Embiid'